In [ ]:
import pypsa
import pandas as pd
import numpy as np

In [ ]:
solver = "cbc"

*** 
Check-point 1:

**In this exercise, we will import the copper-plate power system from Exercise 1 and integrate a storage into the network. If the system does not activate the storage component, we will explore various solutions to enforce storage behavior within the network.**

> **Remarks:** 
> 
> - While the exercises focus on electricity storage, the same concepts can be applied to create storage solutions for other energy carriers. 
> - In these exercises, we will only work with `StorageUnit` component, meaning energy-to-power ratio for storage plant is fixed. To optimize the storage energy capacity independently from the storage power capacity (e.g., in case of hydrogen or gas storage), you should use a fundamental `Store` component in combination with two `Link` components, one for charging and one for discharging (Have a look at this [PyPSA example](https://pypsa.readthedocs.io/en/latest/examples/replace-generator-storage-units-with-store.html)). We will touch on this implementation on day 2 of the training.

Initialize network

In [ ]:
# import the check-point 3 network from exercise 1
n = pypsa.Network("../results/network_d1_e1-3.nc")

In [ ]:
n.generators

In [ ]:
n.loads

Increase `electricity_load` to `200MW` 

In [ ]:
# You can modify network's component values directly via component's DataFrame, but make sure to locate correct index.
load_id = n.loads[n.loads.bus == "electricity"].index
n.loads.loc[load_id, "p_set"] = 200

In [ ]:
n.loads

Add a pumped hydro storage to the network with a fixed energy-to-power ratio of `8 hours`. The rated capacity of the plant should be endogenously decided by the model. All other techno-economic parameters are provided.

> **Source:** all costs for this example are taken from [Danish energy agency technology database for energy storage](https://ens.dk/en/our-services/technology-catalogues/technology-data-energy-storage).

In [ ]:
# We need to calculate annualized capital expenditure
def calculate_annualised_capex(capex: float, interest: float, lifetime: int):
    crf = (
        interest * (1 + interest) ** lifetime / ((1 + interest) ** lifetime - 1)
    )  # Capital recovery factor
    return capex * crf

In [ ]:
# Techno-economic parameters of the pumped hydro storage:
lifetime = 50  # years
interest = 0.05  # unit: -
CAPEX = 600000  # $/MW
FOM = 0.015 * CAPEX  # $/MW per year (between 1% and 2% of the CAPEX)
VOM = 3.9  # $/MWh
fuel_cost = 0  # $/MWh_th per unit water consumed
efficiency_store = 0.8  # assuming similar storing and discharging efficiencies
efficiency_dispatch = 0.8  # assuming similar storing and discharging efficiencies

In [ ]:
annualized_capex = calculate_annualised_capex(CAPEX, interest, lifetime)

n.add(
    class_name="StorageUnit",
    name="pumped_hydro_storage",
    bus="electricity",
    marginal_cost=VOM + fuel_cost,
    capital_cost=annualized_capex + FOM,
    p_nom_extendable=True,
    efficiency_store=efficiency_store,
    efficiency_dispatch=efficiency_dispatch,
    p_max_pu=1,  # Discharging availability
    p_min_pu=-1,  # Charging availability
    max_hours=8,  # energy-to-power ratio
)

Now try to solve the network

In [ ]:
# Solve network using cbc solver
n.optimize(solver_name=solver)

Check if the storage plant is being invested or not?

In [ ]:
n.storage_units["p_nom_opt"]

Check nominal capacities from other technologies

In [ ]:
n.generators["p_nom_opt"]

Ending of check-point 1 - export network

In [ ]:
# Export network
n.export_to_netcdf("../results/network_d1_e2-1.nc")

*** 
Check-point 2:

**How to force investment into storage?**

_TASK: Modify the network to make it invest in either of storage options_

> **Hint: None of the two storage options are invested because storage is not as cost optimal as continue using solar and gas-fired power plants.** 

**Option 1:**

Reducing the cost of storage options to encourage the model to prioritize investing in storage instead of other generation methods.

In [ ]:
# import check-point 1 network from exercise 2
n = pypsa.Network("../results/network_d1_e2-1.nc")

In [ ]:
n.storage_units.loc["pumped_hydro_storage"]

In [ ]:
# Remove capital costs of storage plants
n.storage_units.loc["pumped_hydro_storage", "capital_cost"] = 0

In [ ]:
# Solve network again
n.optimize(solver_name=solver)

Now inspect the invested capacity and dispatch pattern of the system again.

In [ ]:
# Inspect capacity of pumped hydro storage
n.storage_units.loc["pumped_hydro_storage", "p_nom_opt"]

In [ ]:
# Inspect capacity of other plants
n.generators["p_nom_opt"]

In [ ]:
# Inspect storage interaction with other power plants to supply loads
load = n.loads_t.p
pow_gen = n.generators_t.p
storage = n.storage_units_t.p
result = pd.concat([pow_gen, storage, load], axis=1)
result.round().head(24)

In [ ]:
# First drop the load and nuclear generation columns
plot = result.loc[:, ~result.columns.isin(["electricity_load", "nuclear_power_plant"])]
plot.iloc[:48, :].plot(kind="bar", stacked=True, figsize=(14, 5))

**Option 2:**

Having some initial filling for storage

> **Remarks:You can use `state_of_charge_initial` to set initial filling of an storage.**

In [ ]:
# import check-point 1 network from exercise 2
n = pypsa.Network("../results/network_d1_e2-1.nc")

In [ ]:
# Adding initial filling of storage to cover the first 8 hours
load = n.loads.loc["electricity_load", "p_set"]
max_hours = n.storage_units.loc["pumped_hydro_storage", "max_hours"]
n.storage_units.loc["pumped_hydro_storage", "state_of_charge_initial"] = (
    load * max_hours
)

In [ ]:
# Solve network again
n.optimize(solver_name=solver)

Now inspect the invested capacity and dispatch pattern of the system again.

In [ ]:
n.storage_units["p_nom_opt"]

In [ ]:
# Inspect nominal capacity of other plants
n.generators["p_nom_opt"]

In [ ]:
# Inspect storage interaction with other powerplants to supply loads
load = n.loads_t.p
pow_gen = n.generators_t.p
storage = n.storage_units_t.p
result = pd.concat([pow_gen, storage, load], axis=1)

plot = result.loc[
    :, ~result.columns.isin(["electricity_load", "nuclear_power_plant"])
]  # drop load and nuclear columns
plot.iloc[:48, :].plot(kind="bar", stacked=True, figsize=(14, 5))